Upang mapatakbo ang mga sumusunod na notebook, kung hindi mo pa nagagawa, kailangan mong itakda ang openai key sa loob ng .env file bilang `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Susunod, ilalagay natin ang Embedding Index sa isang Pandas Dataframe. Ang Embedding Index ay nakaimbak sa isang JSON file na tinatawag na `embedding_index_3m.json`. Ang Embedding Index ay naglalaman ng mga Embeddings para sa bawat transcript ng YouTube hanggang huling bahagi ng Oktubre 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Susunod, gagawa tayo ng isang function na tinatawag na `get_videos` na hahanapin ang Embedding Index para sa query. Ang function ay magbabalik ng nangungunang 5 video na pinaka-katulad sa query. Ang function ay gumagana sa mga sumusunod na hakbang:

1. Una, isang kopya ng Embedding Index ang ginawa.
2. Sunod, ang Embedding para sa query ay kinakalkula gamit ang OpenAI Embedding API.
3. Pagkatapos, isang bagong kolum ang ginawa sa Embedding Index na tinatawag na `similarity`. Ang kolum na `similarity` ay naglalaman ng cosine similarity sa pagitan ng query Embedding at ng Embedding para sa bawat segment ng video.
4. Sunod, ang Embedding Index ay sinala batay sa kolum na `similarity`. Ang Embedding Index ay sinala upang isama lamang ang mga video na may cosine similarity na mas mataas o katumbas ng 0.75.
5. Sa huli, ang Embedding Index ay inayos ayon sa kolum na `similarity` at ang nangungunang 5 video ay ibinabalik.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Ang function na ito ay napakasimple, ito ay nagpi-print lamang ng mga resulta ng search query.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Una, ang Embedding Index ay niloload sa isang Pandas Dataframe.
2. Sunod, hihilingin sa user na maglagay ng query.
3. Pagkatapos, tatawagin ang `get_videos` function upang maghanap sa Embedding Index para sa query.
4. Sa huli, tatawagin ang `display_results` function upang ipakita ang mga resulta sa user.
5. Pagkatapos, hihilingin muli sa user na maglagay ng isa pang query. Magpapatuloy ang prosesong ito hanggang sa maglagay ang user ng `exit`.

![](../../../../translated_images/tl/notebook-search.1e320b9c7fcbb0bc.webp)

Hihilingin sa iyo na maglagay ng query. Maglagay ng query at pindutin ang enter. Magbabalik ang aplikasyon ng listahan ng mga video na may kaugnayan sa query. Magbabalik din ang aplikasyon ng link sa bahagi ng video kung saan matatagpuan ang sagot sa tanong.

Narito ang ilang mga query na maaari mong subukan:

- Ano ang Azure Machine Learning?
- Paano gumagana ang convolutional neural networks?
- Ano ang neural network?
- Maaari ko bang gamitin ang Jupyter Notebooks kasama ang Azure Machine Learning?
- Ano ang ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Paalala**:
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagamat nagsusumikap kami para sa katumpakan, pakatandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o di-tumpak na impormasyon. Ang orihinal na dokumento sa orihinal nitong wika ang dapat ituring na pangunahing sanggunian. Para sa mahahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
